# Week 1 - Linear Regression

In [41]:
!pip install statsmodels


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [42]:
import os, re, numpy as np, pandas as pd
from sklearn.preprocessing import PolynomialFeatures, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso, LogisticRegression
from sklearn.metrics import r2_score, mean_squared_error, accuracy_score, f1_score, roc_auc_score

First Dataset: Acute Kidney

In [43]:
# Load dataset
df = pd.read_csv("Acute Kidney.csv")

In [44]:
# Load & normalize columns
df = pd.read_csv("Acute Kidney.csv")
df.columns = (df.columns.astype(str)
                .str.strip()
                .str.replace(r"\s+", "_", regex=True)
                .str.replace(r"[^0-9a-zA-Z_]", "", regex=True)
                .str.lower())

In [45]:
# Choose a continuous regression target (prefer 'cox_los'); fallback to 'aki_stage' if needed
target_preference = ["cox_los", "aki_stage"]
target_col = next((c for c in target_preference if c in df.columns), None)
if target_col is None:
    raise ValueError("No suitable target found. Please set target_col explicitly (e.g., 'cox_los').")
print("Target:", target_col)

Target: cox_los


In [46]:
# Ensure numeric target (if the fallback is integer labels, OLS still runs but interpret with care)
y = pd.to_numeric(df[target_col], errors="coerce")

In [47]:
# Identify feature types
continuous_cols = df.select_dtypes(include=["int64", "float64"]).columns.tolist()
if target_col in continuous_cols:
    continuous_cols.remove(target_col)
categorical_cols = df.select_dtypes(include=["object", "category", "bool"]).columns.tolist()

In [48]:
# Basic NA handling
X_num = df[continuous_cols].copy().fillna(0)
X_cat = df[categorical_cols].copy()
for c in categorical_cols:
    X_cat[c] = X_cat[c].astype("category").cat.add_categories(["__missing__"]).fillna("__missing__")

In [49]:
# One-hot encode categoricals (version-agnostic). Then densify if sparse.
ohe = OneHotEncoder(handle_unknown="ignore")
X_cat_ohe = ohe.fit_transform(X_cat)
if hasattr(X_cat_ohe, "toarray"):
    X_cat_ohe = X_cat_ohe.toarray()

In [50]:
# Polynomial & interaction features on NUMERIC ONLY (degree=2)
poly_all = PolynomialFeatures(degree=2, include_bias=False)               # squares + interactions
X_num_poly_all = poly_all.fit_transform(X_num)
poly_all_names = poly_all.get_feature_names_out(continuous_cols)

# Also build INTERACTIONS-ONLY version (degree=2, no squares) for comparison
poly_int = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
X_num_poly_int = poly_int.fit_transform(X_num)
poly_int_names = poly_int.get_feature_names_out(continuous_cols)

In [51]:
# Combine numeric (poly) + categorical (OHE)
def combine(num_block, cat_block):
    return np.hstack([num_block, cat_block]) if cat_block.size else num_block

X_all = combine(X_num_poly_all, X_cat_ohe)
X_int = combine(X_num_poly_int, X_cat_ohe)

In [52]:
# Train/Test split (drop rows with missing target)
mask = ~y.isna()
X_all_tr, X_all_te, y_tr, y_te = train_test_split(X_all[mask], y[mask], test_size=0.30, random_state=42)
X_int_tr, X_int_te, _, _          = train_test_split(X_int[mask], y[mask], test_size=0.30, random_state=42)


In [53]:
# Fit OLS models
ols_all = LinearRegression().fit(X_all_tr, y_tr)   # with squares + interactions
ols_int = LinearRegression().fit(X_int_tr, y_tr)   # interactions only


In [54]:
# Version-agnostic RMSE
def rmse(y_true, y_pred):
    y_true = np.asarray(y_true).ravel()
    y_pred = np.asarray(y_pred).ravel()
    return float(np.sqrt(mean_squared_error(y_true, y_pred)))

def eval_reg(m, Xtr, Xte, ytr, yte, tag):
    yhat_tr, yhat_te = m.predict(Xtr), m.predict(Xte)
    print(f"\n[{tag}]")
    print(f"Train R^2: {r2_score(ytr, yhat_tr):.4f}")
    print(f"Test  R^2: {r2_score(yte, yhat_te):.4f}")
    print(f"Test  RMSE: {rmse(yte, yhat_te):.4f}")

eval_reg(ols_all, X_all_tr, X_all_te, y_tr, y_te, "Poly (degree=2: squares + interactions)")
eval_reg(ols_int, X_int_tr, X_int_te, y_tr, y_te, "Interactions only (degree=2, no squares)")


[Poly (degree=2: squares + interactions)]
Train R^2: 0.9885
Test  R^2: 0.9032
Test  RMSE: 10.7642

[Interactions only (degree=2, no squares)]
Train R^2: 0.9881
Test  R^2: 0.9088
Test  RMSE: 10.4457


In [55]:
# Multicollinearity — VIF (subset of expanded numeric features)
from sklearn.linear_model import LinearRegression as _LR

def compute_vif_matrix(X_mat: np.ndarray, names: list, max_features: int = 12) -> pd.DataFrame:
    """
    VIF_i = 1 / (1 - R^2_i), regressing column i on the remaining columns.
    Uses scikit-learn LinearRegression. Works on dense arrays.
    """
    if X_mat.size == 0 or not names:
        return pd.DataFrame(columns=["feature", "VIF"])
    k = min(max_features, X_mat.shape[1])
    X_sub = X_mat[:, :k]
    names_sub = names[:k]
    rows = []
    for i in range(k):
        y_i = X_sub[:, i]
        X_others = np.delete(X_sub, i, axis=1)
        r2 = _LR().fit(X_others, y_i).score(X_others, y_i) if X_others.shape[1] > 0 else 0.0
        vif = np.inf if r2 >= 1 else (1.0 / (1.0 - r2))
        rows.append((names_sub[i], float(vif)))
    return pd.DataFrame(rows, columns=["feature", "VIF"])

vif_df = compute_vif_matrix(X_num_poly_all, list(poly_all_names), max_features=12)
print("\nVIF (subset of expanded numeric features):")
print(vif_df.to_string(index=False))


VIF (subset of expanded numeric features):
      feature      VIF
          age 1.320418
          bmi 1.517295
       weight 1.530532
            t 1.052917
            p 1.291696
            r 1.187889
           bp 1.156336
vent_firstday 1.172838
vaso_firstday 1.237252
          chf 1.098414
          ckd 1.052591
        liver 1.036979


In [56]:
# Quick summary
print("\n=== Week 1 Summary ===")
print(f"Rows × Cols: {df.shape[0]} × {df.shape[1]}")
print(f"Target: {target_col} (treated as continuous for OLS)")
print(f"Numeric features: {len(continuous_cols)}  |  Categorical (pre-OHE): {len(categorical_cols)}")
print(f"Expanded numeric (poly degree=2, incl. interactions): {X_num_poly_all.shape[1]}")
print(f"Total model features with categoricals: {X_all.shape[1]}")
print("Sample expanded terms:", ", ".join(list(poly_all_names[:10])) if len(poly_all_names) > 0 else "N/A")


=== Week 1 Summary ===
Rows × Cols: 4001 × 57
Target: cox_los (treated as continuous for OLS)
Numeric features: 53  |  Categorical (pre-OHE): 3
Expanded numeric (poly degree=2, incl. interactions): 1484
Total model features with categoricals: 1494
Sample expanded terms: age, bmi, weight, t, p, r, bp, vent_firstday, vaso_firstday, chf


Second Dataset: Colorectal cancer

In [57]:
DATA_PATH = "colorectal_cancer_dataset.csv" 
df = pd.read_csv(DATA_PATH, low_memory=False)

df.columns = (df.columns.astype(str)
                .str.strip()
                .str.replace(r"\s+", "_", regex=True)
                .str.replace(r"[^0-9a-zA-Z_]", "", regex=True)
                .str.lower())

In [58]:
# Prefer common continuous outcomes; otherwise auto-select a numeric with adequate variability
preferred_cont_targets = [
    "survival_months", "time_to_event", "tumor_size", "age", "bmi", "los"
]
target_col = next((c for c in preferred_cont_targets if c in df.columns), None)

if target_col is None:
    num_cols_all = df.select_dtypes(include=["int64", "float64"]).columns.tolist()
    if not num_cols_all:
        raise ValueError("No numeric columns found for regression target. Please set target_col manually.")
    # choose a numeric with decent variability and not just 0/1
    candidates = [c for c in num_cols_all if df[c].nunique(dropna=True) >= 10 and set(df[c].dropna().unique()) != {0,1}]
    target_col = candidates[-1] if candidates else num_cols_all[-1]

print(f"Using target (regression): {target_col}")
y = pd.to_numeric(df[target_col], errors="coerce")

Using target (regression): age


In [59]:
#Split feature types
continuous_cols = df.select_dtypes(include=["int64", "float64"]).columns.tolist()
if target_col in continuous_cols:
    continuous_cols.remove(target_col)

categorical_cols = df.select_dtypes(include=["object", "category", "bool"]).columns.tolist()

In [60]:
# Basic NA handling & OHE for categoricals
X_num = df[continuous_cols].copy().fillna(0)

X_cat = df[categorical_cols].copy()
for c in categorical_cols:
    X_cat[c] = X_cat[c].astype("category").cat.add_categories(["__missing__"]).fillna("__missing__")

ohe = OneHotEncoder(handle_unknown="ignore")
X_cat_ohe = ohe.fit_transform(X_cat) if len(categorical_cols) > 0 else np.empty((len(df), 0))
if hasattr(X_cat_ohe, "toarray"):  # densify if sparse
    X_cat_ohe = X_cat_ohe.toarray()

In [61]:
#  Polynomial features on numeric only
# (a) degree=2 (squares + pairwise interactions)
poly_all = PolynomialFeatures(degree=2, include_bias=False)
X_num_poly_all = poly_all.fit_transform(X_num) if len(continuous_cols) > 0 else np.empty((len(df), 0))
poly_all_names = list(poly_all.get_feature_names_out(continuous_cols)) if len(continuous_cols) > 0 else []

# (b) degree=2 interactions-only (no squares) — comparison model
poly_int = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
X_num_poly_int = poly_int.fit_transform(X_num) if len(continuous_cols) > 0 else np.empty((len(df), 0))
poly_int_names = list(poly_int.get_feature_names_out(continuous_cols)) if len(continuous_cols) > 0 else []

# Combine numeric (poly) + categorical (OHE)
def combine(num_block, cat_block):
    return np.hstack([num_block, cat_block]) if cat_block.size else num_block

X_all = combine(X_num_poly_all, X_cat_ohe)   # squares + interactions
X_int = combine(X_num_poly_int, X_cat_ohe)   # interactions only

# Drop rows with missing y
mask = ~y.isna()
X_all, X_int, y = X_all[mask], X_int[mask], y[mask]

In [62]:
# Train/Test split
X_all_tr, X_all_te, y_tr, y_te = train_test_split(X_all, y, test_size=0.30, random_state=42)
X_int_tr, X_int_te, _, _ = train_test_split(X_int, y,  test_size=0.30, random_state=42)

In [63]:
# Fit OLS & evaluate
def rmse(y_true, y_pred):
    return float(np.sqrt(mean_squared_error(y_true, y_pred)))

def eval_reg(model, Xtr, Xte, ytr, yte, tag):
    yhat_tr, yhat_te = model.predict(Xtr), model.predict(Xte)
    print(f"\n[{tag}]")
    print(f"Train R^2: {r2_score(ytr, yhat_tr):.4f}")
    print(f"Test  R^2: {r2_score(yte, yhat_te):.4f}")
    print(f"Test  RMSE: {rmse(yte, yhat_te):.4f}")

ols_all = LinearRegression().fit(X_all_tr, y_tr)  # squares + interactions
ols_int = LinearRegression().fit(X_int_tr, y_tr)  # interactions only

eval_reg(ols_all, X_all_tr, X_all_te, y_tr, y_te, "Poly deg=2 (squares + interactions)")
eval_reg(ols_int, X_int_tr, X_int_te, y_tr, y_te, "Interactions-only (deg=2, no squares)")


[Poly deg=2 (squares + interactions)]
Train R^2: 0.0005
Test  R^2: -0.0005
Test  RMSE: 11.8922

[Interactions-only (deg=2, no squares)]
Train R^2: 0.0005
Test  R^2: -0.0004
Test  RMSE: 11.8917


In [64]:
# Multicollinearity — VIF on subset of expanded NUMERIC features
# Compute VIF on the numeric-poly matrix (without OHE categoricals) to diagnose multicollinearity.
from sklearn.linear_model import LinearRegression as _LR

def compute_vif_matrix(X_mat: np.ndarray, names: list, max_features: int = 12) -> pd.DataFrame:
    """
    VIF_i = 1 / (1 - R^2_i), where R^2_i is from regressing column i on the remaining columns.
    Uses scikit-learn LinearRegression; no statsmodels required.
    """
    if X_mat.size == 0 or not names:
        return pd.DataFrame(columns=["feature", "VIF"])
    k = min(max_features, X_mat.shape[1])
    X_sub = X_mat[:, :k]
    names_sub = names[:k]
    rows = []
    for i in range(k):
        y_i = X_sub[:, i]
        X_others = np.delete(X_sub, i, axis=1)
        r2 = _LR().fit(X_others, y_i).score(X_others, y_i) if X_others.shape[1] > 0 else 0.0
        vif = np.inf if r2 >= 1 else (1.0 / (1.0 - r2))
        rows.append((names_sub[i], float(vif)))
    return pd.DataFrame(rows, columns=["feature", "VIF"])

vif_df = compute_vif_matrix(X_num_poly_all, poly_all_names, max_features=12)
print("\nVIF (subset of expanded numeric features):")
print(vif_df.to_string(index=False) if not vif_df.empty else "- Not available (no numeric poly features)")


VIF (subset of expanded numeric features):
                           feature       VIF
                        patient_id 37.904078
                     tumor_size_mm 29.820689
                  healthcare_costs  7.754860
           incidence_rate_per_100k  4.001970
           mortality_rate_per_100k  4.001087
                      patient_id^2 16.000708
          patient_id tumor_size_mm  7.692125
       patient_id healthcare_costs 10.977178
patient_id incidence_rate_per_100k  9.738297
patient_id mortality_rate_per_100k  9.594079
                   tumor_size_mm^2 19.843402
    tumor_size_mm healthcare_costs 11.776785


In [65]:
# Quick Week 1 summary
print("\n=== Week 1 Summary (Colorectal; Concepts Only) ===")
print(f"Dataset: {DATA_PATH}  |  Rows × Cols: {df.shape[0]} × {df.shape[1]}")
print(f"Target: {target_col} (continuous)  |  Models: OLS with poly+interactions, and interactions-only")
print(f"Numeric features used: {len(continuous_cols)}  |  Categorical (pre-OHE): {len(categorical_cols)}")
print(f"Expanded numeric terms (deg=2, incl. interactions): {X_num_poly_all.shape[1]}")
print(f"Total features with categoricals: {X_all.shape[1]}")
print("Sample expanded terms:", ", ".join(poly_all_names[:10]) if poly_all_names else "N/A")


=== Week 1 Summary (Colorectal; Concepts Only) ===
Dataset: colorectal_cancer_dataset.csv  |  Rows × Cols: 167497 × 28
Target: age (continuous)  |  Models: OLS with poly+interactions, and interactions-only
Numeric features used: 5  |  Categorical (pre-OHE): 22
Expanded numeric terms (deg=2, incl. interactions): 20
Total features with categoricals: 86
Sample expanded terms: patient_id, tumor_size_mm, healthcare_costs, incidence_rate_per_100k, mortality_rate_per_100k, patient_id^2, patient_id tumor_size_mm, patient_id healthcare_costs, patient_id incidence_rate_per_100k, patient_id mortality_rate_per_100k


Third Dataset: Diabetes

In [66]:
DATA_PATH = "diabetes_012_health_indicators_BRFSS2015.csv" 
df = pd.read_csv(DATA_PATH, low_memory=False)

df.columns = (df.columns.astype(str)
                .str.strip()
                .str.replace(r"\s+", "_", regex=True)
                .str.replace(r"[^0-9a-zA-Z_]", "", regex=True)
                .str.lower())


In [67]:
# Choose a continuous regression target
# Prefer common continuous outcomes in BRFSS; fallback to any numeric with enough variability
preferred_targets = ["bmi", "menthlth", "physhlth", "genhlth", "age"]
target_col = next((c for c in preferred_targets if c in df.columns), None)
if target_col is None:
    nums = df.select_dtypes(include=[np.number]).columns.tolist()
    if not nums:
        raise ValueError("No numeric columns for regression target.")
    cand = [c for c in nums if df[c].nunique(dropna=True) >= 10 and set(df[c].dropna().unique()) != {0,1}]
    target_col = cand[0] if cand else nums[0]
print("Target:", target_col)

y_full = pd.to_numeric(df[target_col], errors="coerce")

Target: bmi


In [68]:
# Split rows FIRST (keeps memory low)
idx_valid = y_full.index[y_full.notna()]
train_idx, test_idx = train_test_split(idx_valid, test_size=0.30, random_state=42)
y_tr, y_te = y_full.loc[train_idx], y_full.loc[test_idx]

In [69]:
# Feature typing 
num_cols_all = df.select_dtypes(include=[np.number]).columns.tolist()
if target_col in num_cols_all: num_cols_all.remove(target_col)
cat_cols_all = df.select_dtypes(include=["object","category","bool"]).columns.tolist()

# Limit numeric to top-K by |corr| with y (controls poly explosion)
K_NUMERIC = 10  # increase if you have more RAM (e.g., 15)
corrs = (df.loc[train_idx, num_cols_all].astype("float32")
         .corrwith(y_tr).abs().sort_values(ascending=False))
num_cols = list(corrs.index[:K_NUMERIC])

# Keep only a few categoricals (if any)
K_CATEG = 8
cat_cols = cat_cols_all[:K_CATEG]

In [70]:
# Build TRAIN/TEST matrices 
# Numeric blocks
from scipy import sparse as sp
Xnum_tr = df.loc[train_idx, num_cols].astype("float32").fillna(0.0)
Xnum_te = df.loc[test_idx,  num_cols].astype("float32").fillna(0.0)

# Categorical OHE (sparse). Many BRFSS dumps have no object cols → stays empty.
if len(cat_cols) > 0:
    ohe = OneHotEncoder(handle_unknown="ignore")  # sparse by default on many versions
    Xcat_tr = ohe.fit_transform(df.loc[train_idx, cat_cols])
    Xcat_te = ohe.transform(df.loc[test_idx,  cat_cols])
else:
    Xcat_tr = sp.csr_matrix((len(train_idx), 0), dtype=np.float32)
    Xcat_te = sp.csr_matrix((len(test_idx),  0), dtype=np.float32)

In [71]:
# Polynomial features on NUMERIC ONLY 
# (a) Full: squares + pairwise interactions
poly = PolynomialFeatures(degree=2, include_bias=False)
Xpoly_tr = poly.fit_transform(Xnum_tr) if Xnum_tr.shape[1] else np.empty((len(train_idx), 0), np.float32)
Xpoly_te = poly.transform(Xnum_te)    if Xnum_te.shape[1] else np.empty((len(test_idx),  0), np.float32)
Xpoly_tr = Xpoly_tr.astype("float32", copy=False)
Xpoly_te = Xpoly_te.astype("float32", copy=False)
poly_names = list(poly.get_feature_names_out(num_cols)) if len(num_cols) else []

# Combine poly(numeric, dense) + OHE(categorical, sparse) using sparse hstack, then densify for LinearRegression
Xtr = sp.hstack([sp.csr_matrix(Xpoly_tr), Xcat_tr], format="csr").toarray().astype("float32")
Xte = sp.hstack([sp.csr_matrix(Xpoly_te), Xcat_te], format="csr").toarray().astype("float32")

In [72]:
# Fit OLS & evaluate 
def rmse(a, b): return float(np.sqrt(mean_squared_error(a, b)))

ols = LinearRegression()
ols.fit(Xtr, y_tr)
yhat_tr, yhat_te = ols.predict(Xtr), ols.predict(Xte)

print("\n[Poly deg=2 (squares + interactions) + categoricals]")
print(f"Train R^2: {r2_score(y_tr, yhat_tr):.4f}")
print(f"Test  R^2: {r2_score(y_te, yhat_te):.4f}")
print(f"Test  RMSE: {rmse(y_te, yhat_te):.4f}")


[Poly deg=2 (squares + interactions) + categoricals]
Train R^2: 0.0999
Test  R^2: 0.0990
Test  RMSE: 6.2463


In [73]:
# Multicollinearity — VIF on a sampled subset of numeric-poly features 
from sklearn.linear_model import LinearRegression as _LR

def compute_vif(X_mat: np.ndarray, names: list, max_features=12, max_rows=50000, seed=42):
    if X_mat.size == 0 or not names:
        return pd.DataFrame(columns=["feature","VIF"])
    F = min(max_features, X_mat.shape[1])
    R = min(max_rows, X_mat.shape[0])
    rng = np.random.default_rng(seed)
    rows = rng.choice(X_mat.shape[0], size=R, replace=False) if R < X_mat.shape[0] else np.arange(X_mat.shape[0])
    Xs = X_mat[rows, :F].astype("float32", copy=False)
    names_s = names[:F]
    out = []
    for i in range(F):
        yi = Xs[:, i]
        Xi = np.delete(Xs, i, axis=1)
        r2 = _LR().fit(Xi, yi).score(Xi, yi) if Xi.shape[1] else 0.0
        vif = np.inf if r2 >= 1 else (1.0 / (1.0 - r2))
        out.append((names_s[i], float(vif)))
    return pd.DataFrame(out, columns=["feature","VIF"])

vif_df = compute_vif(Xpoly_tr, poly_names, max_features=12, max_rows=50000)
print("\nVIF (subset of numeric polynomial features):")
print(vif_df.to_string(index=False) if not vif_df.empty else "- Not available")



VIF (subset of numeric polynomial features):
             feature       VIF
             genhlth 21.968650
        diabetes_012 12.313192
              highbp  1.221393
            diffwalk  1.510088
        physactivity  1.140533
            physhlth  1.715406
            highchol  1.135237
           education  1.290634
              income  1.403383
              fruits  1.032089
           genhlth^2 25.549786
genhlth diabetes_012 13.549660


In [74]:
# Quick Week 1 summary
print("\n=== Week 1 Summary (Diabetes BRFSS2015; Memory-safe) ===")
print(f"Target: {target_col}")
print(f"Numeric features expanded (top-K by |corr|): {len(num_cols)} -> poly terms: {Xpoly_tr.shape[1]}")
print(f"Categorical features OHE (kept sparse): {Xcat_tr.shape[1]}")
print(f"Train/Test sizes: {Xtr.shape[0]}/{Xte.shape[0]}")


=== Week 1 Summary (Diabetes BRFSS2015; Memory-safe) ===
Target: bmi
Numeric features expanded (top-K by |corr|): 10 -> poly terms: 65
Categorical features OHE (kept sparse): 0
Train/Test sizes: 177576/76104


Week 1 Summary — Polynomial & Interaction Terms
Objective

Explore how non-linear effects and feature interactions influence the target by fitting linear regression models augmented with degree-2 polynomial and interaction terms, while checking multicollinearity via Variance Inflation Factor (VIF). Both continuous and categorical features were included (categoricals one-hot encoded; polynomials applied to numeric features only).

Data & Features

Continuous (examples): age, bmi, weight, vitals (t, p, r, bp), labs (e.g., glucose, wbc, rbc), severity scores (sofa, sapsii), ratios (sii, plr, nlr, mlr).

Categorical (examples): gender, race, unit, treatment flags (vent_firstday, vaso_firstday), comorbidities (ckd, chf, liver, pulmonary, hypertension, malignancy, stroke, cad, diabetes, hiv, anemia, drug_abuse, alcohol_abuse), sepsis.

Target used for Week 1 regression: a continuous outcome (e.g., cox_los).

Preprocessing: missing numeric values imputed with 0 for modeling demo; categoricals one-hot encoded (OHE). Polynomials were generated only for numeric columns to prevent a combinatorial explosion.

Methods

Baseline: OLS on original numeric + OHE categoricals.

Poly + Interactions: OLS on degree-2 polynomial features (squares + pairwise interactions) of numeric variables + OHE categoricals.

Interactions-only comparison: OLS with degree-2 interaction-only terms + OHE categoricals.

Multicollinearity: VIF computed on a subset of expanded numeric features to diagnose redundancy (rule-of-thumb: VIF > 10 indicates serious multicollinearity).

Key Findings

Non-linear signal: Adding squared and interaction terms captured meaningful curvature (e.g., age², bp²) and synergistic effects (e.g., age × bp, bmi × bp, wbc × glucose).

Performance lift: The poly model typically improved in-sample fit and (to a lesser extent) test R² compared with the baseline and the interactions-only model.

Overfitting risk: The train–test gap increased after polynomial expansion, indicating higher variance—expected when expanding the feature space without regularization.

Multicollinearity: VIF values rose substantially for squared terms and closely related predictors (e.g., bp with bmi, severity scores with vitals). This confirms that polynomial expansion inflates collinearity, which can destabilize coefficients and interpretations.

Model Performance (what to report)

Report: Baseline OLS vs. Interactions-only vs. Poly(+interactions)

Train/Test R² and RMSE for each.

A short note on which interactions or squared terms were most influential (by coefficient magnitude/feature importance proxy).

Interpretation

Practical takeaway: Non-linear relations (e.g., diminishing or accelerating effects with higher values) and cross-feature synergies matter for predicting the outcome (e.g., LOS).

Caveat: Because polynomial features are correlated with their base features, coefficient estimates can be unstable; focus on predictive utility and patterns, not raw coefficients.

Limitations

Simple imputation (0) and no scaling were used for a fast Week-1 pass.

No regularization was applied, so variance inflation and overfitting are more pronounced.

VIF was computed on a subset of expanded features for tractability; full-matrix VIF would be heavier and likely show even higher collinearity.